In [13]:
from IPython import get_ipython
from IPython.display import display
# %%
import random
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from langchain_ollama.llms import OllamaLLM
# Initialize LLM
llm = OllamaLLM(model="llama3.2")

# Agent class
class LLMAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.llm = llm  # Access the LLM instance

    def make_decision(self, context):
        prompt = f"Given the context: {context}, make a decision based on your goals."
        response = self.llm.invoke(prompt)
        decision = str(response)
        print(f"Agent {self.unique_id} made decision: {decision}")
        return decision

    # def step(self):
    #     context = f"Agent {self.unique_id} is deciding what to do next."
    #     decision = self.make_decision(context)
    #     print(f"Agent {self.unique_id} made decision: {decision}")

# Model class
class LLMModel(Model):
    def __init__(self, width, height, n):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)

        # Create agents
        agents = LLMAgent.create_agents(model=self, n=n)
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

        # Data collector for tracking decision-making
        self.datacollector = DataCollector(
            agent_reporters={"Decision": lambda a: a.make_decision(f"Agent {a.unique_id} is deciding what to do next.")}
        )
        

    def step(self):
        # self.datacollector.collect(self)
        context = """
You are stuck on an island, you have 3 choices:
1) Search for fruits
2) Search for material for shelter
3) Look around the island for the closest ship to contact
Which option would you pick (1,2,3) ?
"""
        self.agents.shuffle_do("make_decision", context)

# Run the model
def run_model():
    model = LLMModel(10, 10, 5)
    for i in range(10):
        print(f"Step {i+1}")
        model.step()
    # agent_data = model.datacollector.get_agent_vars_dataframe()
    # print(agent_data)

if __name__ == "__main__":
    run_model()

Step 1
Agent 4 made decision: Given my goal is to survive and potentially get rescued as soon as possible, I would choose option 3: Look around the island for the closest ship to contact.

Finding a ship with a means of communication could be the quickest way to get help and possibly even rescue. Searching for fruits (option 1) might provide sustenance in the short term but wouldn't address my primary goal of getting off the island. Building shelter (option 2) is also important, but it's not as crucial as finding a means of communication to alert others about my situation.

By prioritizing option 3, I'm taking a proactive step towards potentially getting rescued sooner rather than later.
Agent 3 made decision: A classic survival scenario!

Given my goal is to survive and potentially get rescued, I would choose:

**Option 3: Look around the island for the closest ship to contact**

Why? Well, finding shelter and food are essential, but they don't necessarily guarantee rescue. On the oth

KeyboardInterrupt: 

In [1]:
from mesa import Agent, Model
import random
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from langchain_ollama.llms import OllamaLLM

# Initialize LLM
llm = OllamaLLM(model="llama3.2")

# Agent class with Personality
class LLMAgent(Agent):
    def __init__(self, unique_id, model, personality):
        # Correct initialization of Agent, with the model being passed as the first argument.
        super().__init__(model)  # Pass unique_id first and then model to the Agent base class
        self.llm = llm  # Access the LLM instance
        self.personality = personality  # Personality affects decision-making
        self.stock_held = 0  # Start with no stock holdings
        self.money_held = 100

    def make_decision(self, context):
        prompt = f"Personality: {self.personality}. Context: {context}. Make a decision based on your personality and goals."
        response = self.llm.invoke(prompt)
        decision = str(response)
        print(f"Agent {self.unique_id} made decision: {decision}")
        return decision

    def buy_stock(self, amount):
        self.stock_held += amount
        print(f"Agent {self.unique_id} bought {amount} of Banana stock.")

    def sell_stock(self, amount):
        if self.stock_held >= amount:
            self.stock_held -= amount
            print(f"Agent {self.unique_id} sold {amount} of Banana stock.")
        else:
            print(f"Agent {self.unique_id} tried to sell more than they own.")

    def hold_stock(self):
        print(f"Agent {self.unique_id} decided to hold their Banana stock.")


# Model class with Stock Pricing
class LLMModel(Model):
    def __init__(self, width, height, n, initial_price=100):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)
        self.banana_price = initial_price  # Initial price of the "banana" stock
        self.buy_count = 0  # Track the number of buys
        self.sell_count = 0  # Track the number of sells

        # Create agents with different personalities
        personalities = ["Aggressive", "Cautious", "Risk-Averse", "Optimistic", "Pessimistic"]
        agents = [LLMAgent(i, self, random.choice(personalities)) for i in range(self.num_agents)]
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

        # Data collector for tracking decision-making
        self.datacollector = DataCollector(
            agent_reporters={"Decision": lambda a: a.make_decision(f"Agent {a.unique_id} is deciding what to do next.")},
            model_reporters={"BananaPrice": "banana_price"}
        )

    def calculate_stock_price(self):
        # Simple supply-demand model for stock price
        price_change = (self.buy_count - self.sell_count) * 0.1  # Small price fluctuation based on buy/sell activity
        self.banana_price += price_change
        self.banana_price = max(1, self.banana_price)  # Ensure the price doesn't go below 1
        print(f"New Banana stock price: {self.banana_price}")

    def step(self):
        self.buy_count = 0
        self.sell_count = 0
        
        context = """
The current price of Banana stock is ${}. 
You have the following options:
1) Buy Banana stock.
2) Sell Banana stock.
3) Hold your stock.

Make your decision. Be mindful of your personality and the current market conditions. Start your response with the number of the option chosen and then share your reasoning. Kepp the reasoning brief.
Response: 
""".format(self.banana_price)

        # Agents decide on their actions
        for agent in self.agents:
            decision = agent.make_decision(context)
            if "1)" in decision.lower():
                agent.buy_stock(1)
                self.buy_count += 1
            elif "2)" in decision.lower():
                agent.sell_stock(1)
                self.sell_count += 1
            else:
                agent.hold_stock()

        # Update the stock price after all actions
        self.calculate_stock_price()
        self.datacollector.collect(self)

# Run the model
def run_model():
    model = LLMModel(10, 10, 5)
    for i in range(10):
        print(f"Step {i+1}")
        model.step()

if __name__ == "__main__":
    run_model()


c:\Users\m\Documents\GitHub\GSoC-Project-Mesa-Proposal\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 1
Agent 1 made decision: 1) Sell Banana stock.

I'm inclined to sell due to my pessimistic nature, which often clouds my investment decisions. I don't expect the price of Banana stock to drop significantly, but I also don't think it will increase substantially in the near future. By selling now, I'll at least avoid taking on more risk than necessary and potentially be left with a loss that aligns with my expectations of a disappointing outcome.
Agent 1 bought 1 of Banana stock.
Agent 2 made decision: 1. Sell Banana stock.

I'm not optimistic about the future price of Banana stock, considering its current low value of $100. Given my pessimistic nature, I believe the market is likely to fluctuate downwards, and it's better to cash out now rather than risking a potential loss by holding onto the stock. Selling will allow me to cut my losses and move on from this investment.
Agent 2 decided to hold their Banana stock.
Agent 3 made decision: 1. Sell Banana stock.

I'm choosing to sell 

KeyboardInterrupt: 